In [0]:
%sql
select * from hive_metastore.bronze_incremental_schema.arancione
where lastUpdateDate = (select max(lastUpdateDate) from hive_metastore.bronze_incremental_schema.arancione)

In [0]:
%sql
select distinct * from hive_metastore.silver_incremental_schema.celeste

In [0]:
from pyspark.sql.functions import lit,current_timestamp
from pyspark.sql.types import TimestampType

In [0]:
entity_name = 'Arancione_incremental'

tgt_df = spark.read.table(f"hive_metastore.silver_incremental_schema.{entity_name.split('_inc')[0]}").filter('end_date is null').filter('ArancioneID=2')

src_df = spark.read.table(f"hive_metastore.bronze_incremental_schema.{entity_name.split('_inc')[0]}").withColumnRenamed("lastUpdateDate", "sourceLastUpdateDate").filter('ArancioneID=2').withColumn('Score',lit(100))

scd2_cond = " OR ".join(
            [
                f"target.{u_col} != source.{u_col}"
                for u_col in src_df.columns
                if u_col not in ["processing_dttm"] + ['ArancioneID']
            ]
        )

tgt_df.registerTempTable('target_table')
src_df.registerTempTable('source_table')

src_df.display()
tgt_df.display()
query =   f'''
    select source.* from source_table source inner join target_table target on source.ArancioneID=target.ArancioneID and
    ({scd2_cond})
    '''


ingest_df = spark.sql(query)
ingest_df = ingest_df.withColumn('start_date',current_timestamp())
ingest_df = ingest_df.withColumn("end_date", lit(None).cast(TimestampType()))

ingest_df = ingest_df.union(tgt_df)
display(ingest_df)

# ingest_df.write.mode("append").saveAsTable(f"hive_metastore.silver_incremental_schema.{self.entity_name.split('_inc')[0]}")

In [0]:
%sql
drop table hive_metastore.silver_incremental_schema.arancione

In [0]:
spark.sql('select CURRENT_TIMESTAMP as current_ts').collect()[0][0].isoformat()